## Filter
One of the most powerful parameters when defining a search is the ability to filter the results.  While a Query is more of an unstructured approach to search for text within documents, a Filter provides a well-defined structured approach when we know which properties we want to restrict our matching documents.

The following examples will be using some of the popular properties as outlined within the "Properties & Metadata" notebook.  In addition, the selection of these properties was determined based on the discovery techniques used such as metadata and debugging.

In [4]:
import refinitiv.data as rd
from refinitiv.data.content import search
import pandas as pd
import datetime as dt
import dateutil.relativedelta

# Default session - desktop
rd.open_session()

<refinitiv.data.session.Definition object at 0x25e7fd93820 {name='workspace'}>

In [5]:
pd.set_option('display.max_colwidth', 140)
rd.__version__

'1.1.0'

#### Example - Query vs Filter
As you become more familiar with Search, you will realize there are a number of different paths you can take to arrive at the same destination.

In [7]:
# Retrieve the document(s) for a specific ISIN using a Query.
rd.discovery.search(
    view = search.Views.INSTRUMENTS,
    query = 'US046353AQ14',
    select = 'DocumentTitle, RIC, ISIN, CUSIP, SEDOL, PermID'
)

,DocumentTitle,ISIN,CUSIP,SEDOL,PermID
0,"AstraZeneca PLC, Plain Vanilla Fixed Coupon Bond, TRACE Eligible AZN 2.375 12-Jun-2022",US046353AQ14,046353AQ1,BF4SNM7,46645702869


In [9]:
# Retrieve the document(s) for the same ISIN using a Filter
rd.discovery.search(
    view = search.Views.INSTRUMENTS,
    filter = "ISIN eq 'US046353AQ14'",
    select = 'DocumentTitle, RIC, ISIN, CUSIP, SEDOL, PermID'
)

,DocumentTitle,ISIN,CUSIP,SEDOL,PermID
0,"AstraZeneca PLC, Plain Vanilla Fixed Coupon Bond, TRACE Eligible AZN 2.375 12-Jun-2022",US046353AQ14,046353AQ1,BF4SNM7,46645702869


In [10]:
# If you are provided a code but are not sure if the code is an ISIN, SEDOL, CUSIP etc, 
# try using the 'CodeResolutionBase' property.
rd.discovery.search(
    view = search.Views.INSTRUMENTS,
    filter = "CodeResolutionBase eq '046353AQ1'",   # I'm using the CUSIP for the same instrument
    select = 'DocumentTitle, RIC, ISIN, CUSIP, SEDOL, PermID'
)

,DocumentTitle,ISIN,CUSIP,SEDOL,PermID
0,"AstraZeneca PLC, Plain Vanilla Fixed Coupon Bond, TRACE Eligible AZN 2.375 12-Jun-2022",US046353AQ14,046353AQ1,BF4SNM7,46645702869


#### Example - Apple Equities

In [11]:
# Within the view (EquityQuotes), select all documents where the term 'AAPL' is within the ticker symbol.  For each, 
# ensure the type of asset is an 'equity' and that each asset is in an active state.  I chose to filter out the 
# AssetState of 'DC' (De-activated) as my criteria. By default, the top 10 results are presented.
#
# Note: An alternative to "AssetType eq 'equity'" could be to use: "RCSAssetCategoryLeaf eq 'Ordinary Share'"
rd.discovery.search(
    view = search.Views.EQUITY_QUOTES,
    filter = "AssetType eq 'equity' and AssetState ne 'DC' and TickerSymbol eq 'AAPL'",
    select = "RIC, CompositeDescriptiveName, ExchangeCode, ExchangeName, AssetType"
)

,RIC,CompositeDescriptiveName,ExchangeCode,ExchangeName,AssetType
0,AAPL.O,Apple Ord Shs APPLE ORD,NSQ,NASDAQ Global Select Consolidated,[EQUITY]
1,AAPL.OQ,Apple Ord Shs APPLE ORD,NSM,NASDAQ Stock Exchange Global Select Market,[EQUITY]
2,AAPL.DG,Apple Ord Shs APPLE ORD,GCD,Cboe EDGX Exchange - Nasdaq Global Select Market,[EQUITY]
3,AAPL.Z,Apple Ord Shs APPLE ORD APPLE INC,GCZ,Cboe BZX Exchange - Nasdaq Global Select Market,[EQUITY]
4,AAPL.B,Apple Ord Shs APPLE ORD,GSB,Boston SE when trading NASDAQ Global Select Market,[EQUITY]
5,AAPL.CCP,APPLE CAD HEDGED CDR Apple CAD Hedged CDR,CAQ,Canadian Composite Quote/Trade,[EQUITY]
6,AAPLE.MI,Apple Ord Shs APPLE ORD,MIL,Milan Stock Exchange,[EQUITY]
7,AAPL.DY,Apple Ord Shs APPLE ORD,GDA,Cboe EDGA Exchange - Nasdaq Global Select Market,[EQUITY]
8,AAPL.PH,Apple Ord Shs APPLE ORD APPLE INC,GSX,NASDAQ OMX PHLX Global Select Market,[EQUITY]
9,AAPL.ZY,Apple Ord Shs APPLE ORD APPLE INC,GCY,Cboe BYX Exchange - Nasdaq Global Select Market,[EQUITY]


#### Example - Vessels
List all Very Large Gas Containers (VLGC) that are in service within the last week.

In [12]:
# Build date range expression for the location of the vessel for the last 7 days
date_7_days_ago = dt.datetime.today() - dateutil.relativedelta.relativedelta(days=7)
date_range = f'LocationTimestamp ge {date_7_days_ago.strftime("%Y-%m-%d")} and \
               LocationTimestamp lt {dt.datetime.today().strftime("%Y-%m-%d")}'

In [13]:
# Show the location of each vessel and some general details such as location, destination and coordinates.
# If available, show the top 100 hits.
rd.discovery.search(
    view = search.Views.VESSEL_PHYSICAL_ASSETS,
    filter = f"RCSAssetTypeLeaf eq 'VLGC' and VesselStatus eq 'IN SERVICE' and {date_range}",
    select = 'RIC, SynonymSearchAll, DTSource, LocationTimestamp, IMO,Latitude, Longitude, LocationDestination',
    top=100
)

,RIC,SynonymSearchAll,DTSource,LocationTimestamp,IMO,Latitude,Longitude,LocationDestination
0,C}KY7309484995,Indian India Liquefied Petroleum Gas Very Large Gas Carrier,Arabia Sea,2023-02-19T12:29:16.000Z,9264908,18.850416,64.966377,RUWAIS [Jebel Dhanna/Ruwais]
1,C}KY7309693658,Japanese Liquefied Petroleum Gas Very Large Gas Carrier,Pacific Ocean (Central America)|Hawaii Tanker Zone,2023-02-19T01:45:59.000Z,9723679,20.646778,-170.770477,PA BLB [Balboa]
2,C}KY7309843282,Chinese Liquefied Petroleum Gas Very Large Gas Carrier,South China Sea|Canton Tanker Zone,2023-02-20T22:58:28.000Z,9800506,21.522936,118.206093,CNQZH [Qinzhou]
3,C}KY7309432434,Chinese Vietnamese Liquefied Petroleum Gas Very Large Gas Carrier,South China Sea|Canton Tanker Zone,2023-02-20T22:52:54.000Z,7420089,20.458866,106.725937,HAI HA BUOY


#### Example - Floating Rate Notes
Retrieve all floating rate notes (floating coupon) in the healthcare sector which are issued in Euros.  For each, ensure the coupon rate is great than 2 and our Moody's rating is greater than 'Baa1'.

To determine the rating, we apply the following logic utilizing the RatingsScope property - which is a nested property providing additional details:

- RatingsScope(RatingType eq 'MIS' and CurrentRatingRank lt 10)

The RatingType of 'MIS' refers to the "Moody's Long-term Issuer Rating".
The CurrentRatingRank < 10 instructs search to look for ratings less than 10 ('Baa1').  **Note**: The lower the rank, the better the rating.

In [14]:
rd.discovery.search(
    view = search.Views.GOV_CORP_INSTRUMENTS,
    filter = "CouponRate gt 2 and \
              RatingsScope(RatingType eq 'MIS' and CurrentRatingRank lt 10) and \
              RCSCouponTypeName eq 'Floating Coupon' and \
              IsActive eq true and \
              RCSTRBC2012Name eq 'Healthcare' and \
              RCSCurrencyLeaf xeq 'Euro'",
    select = "CouponCurrency, IssuerName, IssueDate, RIC, CouponRate, ISIN, Ticker, RCSTRBC2012Leaf, MaturityDate, AssetState",   
)

,CouponCurrency,IssuerName,IssueDate,RIC,CouponRate,ISIN,Ticker,RCSTRBC2012Leaf
0,EUR,Sanofi FR,1983-07-01,FR0000140022=,15.5,FR0000140022,SASY,Pharmaceuticals (NEC)
1,EUR,Sanofi FR,1987-10-01,FR0000047656=MFID,15.5,FR0000047656,SASY,Pharmaceuticals (NEC)


#### Example - Outstanding Green Bonds
List the outstanding Green Bonds for Telefonica.

In [15]:
rd.discovery.search(
    view = search.Views.GOV_CORP_INSTRUMENTS,
    filter = "IssuerName eq 'Telefonica' and IsGreenBond eq true",
    select = "IssuerName, DocumentTitle, RIC, CouponRate, ISIN, IssuerTicker, DebtTypeDescription, EOMAmountOutstanding, FaceOutstanding"
)

,IssuerName,DocumentTitle,RIC,CouponRate,ISIN,IssuerTicker,DebtTypeDescription,EOMAmountOutstanding,FaceOutstanding
0,Telefonica Europ,"Telefonica Europe BV, Bond, TEF 2.502",ES210981985=,2.502,XS2109819859,TEFBV,Junior Subordinated Note,500000000,500000000
1,Telefonica Europ,"Telefonica Europe BV, Bond, TEFBV 7.125",ES246260567=,7.125,XS2462605671,TEFBV,Junior Subordinated Note,750000000,750000000
2,Telefonica Europ,"Telefonica Europe BV, Bond, TEFBV 6.135",ES258238915=,6.135,XS2582389156,TEFBV,Capital Security,1000000000,1000000000
3,Telefonica Emis,"Telefonica Emisiones SAU, Plain Vanilla Fixed Coupon Bond, TEF 1.069 05-Feb-2024",ES194600445=,1.069,XS1946004451,TEFEMI,Senior Note,1000000000,1000000000


#### Example - Equity Index listing in the U.S.A

In [16]:
# Based on the Asset Category (Equity Index), list the top 10 indexes within the USA.
rd.discovery.search(
    view = search.Views.INDEX_INSTRUMENTS,
    filter = "RCSIndexCountryGroupLeaf xeq 'United States' and RCSAssetCategoryLeaf xeq 'Equity Index'",
    select = "DocumentTitle, RIC, RCSTRBC2012Leaf"
)

,DocumentTitle,RIC,RCSTRBC2012Leaf
0,"NASDAQ 100 Index, Equity Index",.NDX,UK Investment Trusts
1,"NASDAQ Composite Index, Equity Index",.IXIC,Financial & Commodity Market Operators & Service Providers (NEC)
2,"Dow Jones Industrial Average Index, Equity Index",.DJI,Consumer Publishing (NEC)
3,"Refinitiv United States Price Return Index, Equity Index",.TRXFLDUSP,Professional Information Services (NEC)
4,"S&P 500 Index - CBOE, Equity Index",.SPX,Magazine Publishing
5,"NYSE Composite Index, Equity Index",.NYA,Financial & Commodity Market Operators & Service Providers (NEC)
6,"Dow Jones Transport Average Index, Equity Index",.DJT,Consumer Publishing (NEC)
7,"Dow Jones Composite Index, Equity Index",.DJA,Consumer Publishing (NEC)
8,"Dow Jones Utility Average Index, Equity Index",.DJU,Consumer Publishing (NEC)
9,"Philadelphia SE Semiconductor Index, Equity Index",.SOX,Securities & Commodity Exchanges


#### Example - Search for Brazil global sovereign bonds in USD with a duration greater than 1

In [17]:
rd.discovery.search(
    view=search.Views.FIXED_INCOME_INSTRUMENTS,
    filter = "RCSIssuerCountryLeaf eq 'Brazil' and \
              RCSCurrencyLeaf eq 'US Dollar' and \
              RCSTRBC2012Leaf eq 'Government & Government Finance' and \
              OptionAdjustedDuration gt 1",
    select ="RIC, ISIN, MaturityDate, CouponRate, IndustrySector, FaceOutstandingUSD, OptionAdjustedDuration"
)

,RIC,ISIN,MaturityDate,CouponRate,IndustrySector,FaceOutstandingUSD,OptionAdjustedDuration
0,BR012631375=,US105756AR10,2024-04-15,8.875,SOVERGRN,1029030000,1.055949
1,BR017244388=,US105756AZ36,2024-04-15,8.875,SOVERGRN,85557000,1.055949
2,105756BV1=,US105756BV13,2025-01-07,4.25,SOVERGRN,4300000000,1.765833
3,BR021210641=,US105756BF62,2025-02-04,8.75,SOVERGRN,688201000,1.785662
4,105756CD0=,US105756CD06,2025-06-06,2.875,SOVERGRN,1750000000,2.156112
5,105756BX7=,US105756BX78,2026-04-07,6.0,SOVERGRN,2176245000,2.76905
6,105756AE0=,US105756AE07,2027-05-15,10.125,SOVERGRN,765023000,3.426415
7,105756BZ2=,US105756BZ27,2028-01-13,4.625,SOVERGRN,3000000000,4.216577
8,105756CA6=,US105756CA66,2029-05-30,4.5,SOVERGRN,2000000000,5.250205
9,BR010862957=,US105756AL40,2030-03-06,12.25,SOVERGRN,237592000,4.894292


In [18]:
# Alternatively, you have the option of trying it this way which will render the same results
# using alternative filter properties, i.e. IssuerCountryName, IndustrySectorDescrioption and Currency
rd.discovery.search(
    view=search.Views.FIXED_INCOME_INSTRUMENTS,
    filter = "IssuerCountryName eq 'Brazil' and \
              OptionAdjustedDuration gt 1 and \
              Currency eq 'USD' and IndustrySectorDescription eq 'Sovereign'",
    select ="RIC, ISIN, MaturityDate, CouponRate,IndustrySector, FaceOutstandingUSD, OptionAdjustedDuration"
)

,RIC,ISIN,MaturityDate,CouponRate,IndustrySector,FaceOutstandingUSD,OptionAdjustedDuration
0,BR012631375=,US105756AR10,2024-04-15,8.875,SOVERGRN,1029030000,1.055949
1,BR017244388=,US105756AZ36,2024-04-15,8.875,SOVERGRN,85557000,1.055949
2,105756BV1=,US105756BV13,2025-01-07,4.25,SOVERGRN,4300000000,1.765833
3,BR021210641=,US105756BF62,2025-02-04,8.75,SOVERGRN,688201000,1.785662
4,105756CD0=,US105756CD06,2025-06-06,2.875,SOVERGRN,1750000000,2.156112
5,105756BX7=,US105756BX78,2026-04-07,6.0,SOVERGRN,2176245000,2.76905
6,105756AE0=,US105756AE07,2027-05-15,10.125,SOVERGRN,765023000,3.426415
7,105756BZ2=,US105756BZ27,2028-01-13,4.625,SOVERGRN,3000000000,4.216577
8,105756CA6=,US105756CA66,2029-05-30,4.5,SOVERGRN,2000000000,5.250205
9,BR010862957=,US105756AL40,2030-03-06,12.25,SOVERGRN,237592000,4.894292


#### Example - Top executives

In [19]:
# Using the People View, we can utilize the OfficerDirector property, which contains a number of nested or sub-properties 
# that allow me to perform a more fine-grained search against active people that have been assigned a title code of 
# CEO (Chief Executive Officer). The results are listed based on a default rank defined by the search service.
rd.discovery.search(
    view = search.Views.PEOPLE,
    filter = "OfficerDirector(RoleTitleCode eq 'CEO' and RoleStatus eq 'Active')",
    top = 20,
    select = "FullName, PrimaryTitleAsReported, PrimaryOfficerDirector, RIC, OfficerDirectorActiveCompanies, PrimaryOfficerDirectorRIC"
)

,FullName,PrimaryTitleAsReported,PrimaryOfficerDirector,OfficerDirectorActiveCompanies,PrimaryOfficerDirectorRIC
0,Satya Nadella,"Chairman of the Board, Chief Executive Officer",Microsoft Corp,"[Starbucks Corp, Chandler Insurance Company Ltd, Microsoft Corp]",MSFT.O
1,Tim D. Cook,"Chief Executive Officer, Director",Apple Inc,"[Apple Inc, Nike Inc]",AAPL.O
2,Andrew R. Jassy,"President, Chief Executive Officer, Director",Amazon.com Inc,"[Coupa Software Inc, Amazon.com Inc]",AMZN.O
3,Sundar Pichai,"Chief Executive Officer, Director",Alphabet Inc,"[Alphabet Inc, Ruba Inc (Fka Kudo Inc)]",GOOGL.O
4,R. Martin Chavez,Director,Alphabet Inc,"[Alphabet Inc, Quorum Software Systems Inc, Kiodex Inc]",GOOGL.O
5,Mark Zuckerberg,"Chairman of the Board, Chief Executive Officer, Founder",Meta Platforms Inc,"[Chan Zuckerberg Initiative LLC, Meta Platforms Inc]",META.O
6,Warren E. Buffett,"Chairman of the Board, Chief Executive Officer",Berkshire Hathaway Inc,"[Grinnell College, Bill & Melinda Gates Foundation Trust, Berkshire Hathaway Inc, Kraft Heinz Intermediate Corporation II, Kraft Heinz F...",BRKa
7,Daniel Zhang,"Chairman of the Board, Chief Executive Officer",Alibaba Group Holding Ltd,[Alibaba Group Holding Ltd],BABA.K
8,Huateng Ma,"Executive Chairman of the Board, Chief Executive Officer",Tencent Holdings Ltd,"[Huarong Tianze Investment Co Ltd, Tencent Holdings Ltd, Tencent Technology Beijing Co Ltd]",0700.HK
9,Ryan McInerney,"Chief Executive Officer, Director",Visa Inc,[Visa Inc],V


#### Example - Live Options
The following example will specify an underlying instrument (RIC) and pull the list of options that are active, i.e. not expired.

In [22]:
# Define our underlying
underlying = "'AAPL.O'"

# To search for our instrument, I'm utilizing the property: 'UnderlyingQuoteRIC'.  The property does not support an 
# exact match 'xeq', so we must use the partial match 'eq' to find all hits. Depending on the instrument you specify, 
# this may introduce additional hits.  For example, if you are using the underlying 'AAPL.O', search will also find a 
# hit on 'AAPL.OQ', for example. To protect against this, we can utilize a combination of the built-in functions, 
# 'startswith' and 'endwith' to perform our match.
rd.discovery.search(
    view = search.Views.EQUITY_DERIVATIVE_QUOTES,
    filter = "AssetState eq 'AC' and \
              startswith(UnderlyingQuoteRIC, " + underlying + ") and \
              endswith(UnderlyingQuoteRIC," + underlying + ") and \
              RCSAssetClass eq 'OPT' and IsChain eq false and \
              ExpiryDate ge " + dt.datetime.today().strftime('%Y-%m-%d'),
    select = "DocumentTitle, RIC, UnderlyingQuoteRIC, ExpiryDate, CallPutOption, StrikePrice",
    top = 10000
)

,DocumentTitle,RIC,UnderlyingQuoteRIC,ExpiryDate,CallPutOption,StrikePrice
0,"OPRA Apple Equity Option 150 Call Mar 2023 , Equity Cash Option, Call 150 USD 17-Mar-2023, OPRA",AAPLC172315000.U,[AAPL.O],2023-03-17,Call,150.0
1,"OPRA Apple Equity Option 155 Call Mar 2023 , Equity Cash Option, Call 155 USD 17-Mar-2023, OPRA",AAPLC172315500.U,[AAPL.O],2023-03-17,Call,155.0
2,"OPRA Apple Equity Option 160 Call Mar 2023 , Equity Cash Option, Call 160 USD 17-Mar-2023, OPRA",AAPLC172316000.U,[AAPL.O],2023-03-17,Call,160.0
3,"OPRA Apple Equity Option 145 Put Mar 2023 , Equity Cash Option, Put 145 USD 17-Mar-2023, OPRA",AAPLO172314500.U,[AAPL.O],2023-03-17,Put,145.0
4,"OPRA Apple Equity Option 165 Call Mar 2023 , Equity Cash Option, Call 165 USD 17-Mar-2023, OPRA",AAPLC172316500.U,[AAPL.O],2023-03-17,Call,165.0
...,...,...,...,...,...,...
2319,"CBOE Apple PM Settled European Flex Equity option 185 Call Apr 2023 , Equity Cash Option, Call 185 USD 27-Apr-2023, Chicago Board Option...",2AAPLD2723D00185000.FO,[AAPL.O],2023-04-27,Call,185.0
2320,"CBOE Apple PM Settled European Flex Equity option 150 Call Apr 2023 , Equity Cash Option, Call 150 USD 21-Apr-2023, Chicago Board Option...",2AAPLD2123D00150000.FO,[AAPL.O],2023-04-21,Call,150.0
2321,"CBOE Apple PM Settled European Flex Equity option 150 Put Apr 2023 , Equity Cash Option, Put 150 USD 21-Apr-2023, Chicago Board Options ...",2AAPLP2123D00150000.FO,[AAPL.O],2023-04-21,Put,150.0
2322,"CBOE Apple PM Settled European Flex Equity option 145.01 Call Apr 2023 , Equity Cash Option, Call 145.01 USD 21-Apr-2023, Chicago Board ...",2AAPLD2123D00145010.FO,[AAPL.O],2023-04-21,Call,145.01


In [23]:
rd.close_session()